# Model Optimization

## Optimization Attempt 1 Outlier Insensitivity
We will test our model using two new loss functions since decreasing the output of the loss function can have an input on how outliers effect the accuracy of your model.
The functions in contention are: Hinge Loss , KL Divergance , Poisson Class , and Binary Cross Entropy

## Preprocessing

In [3]:
# importing the PreProcessing function and pandas
from model import PreProcess
from model import PreProcess
filepath = 'Resources/charity_data.csv'
dropCols = ['EIN' , 'NAME']
thresholds = [500 , 1000]

output = PreProcess(filepath , dropCols , thresholds)



from sklearn.model_selection import train_test_split
# Splitting data into training and testing sets
X_train , X_test , y_train , y_test = train_test_split(output[1] , output[0] , random_state= 78)

from sklearn.preprocessing import StandardScaler
# scaling the data sets
scaler = StandardScaler()

# Fitting the scaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


## Training

In [13]:
import tensorflow as tf
# Creating our neural network
nn_model = tf.keras.models.Sequential()

# Creating our first Dense Layer
nn_model.add(tf.keras.layers.Dense(units=43 ,activation= 'relu' , input_dim = 43 )) # there are 43 features that will serve as inputs

# Adding our second layer
nn_model.add(tf.keras.layers.Dense(units=43 , activation= 'relu' , input_dim = 43))

# Creating the output layer
nn_model.add(tf.keras.layers.Dense(units = 1 , activation = 'sigmoid'))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 43)                1892      
_________________________________________________________________
dense_1 (Dense)              (None, 43)                1892      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 44        
Total params: 3,828
Trainable params: 3,828
Non-trainable params: 0
_________________________________________________________________


In [14]:
# Compiling  the model
nn_model.compile(loss = 'poisson' , optimizer = 'adam' , metrics = ['accuracy'])

# Fit the model to the training data
fit_model = nn_model.fit(X_train_scaled , y_train , epochs = 100)

Epoch 1/100
804/804 [==============================] - 1s 943us/step - loss: 0.8084 - accuracy: 0.7181
Epoch 2/100
804/804 [==============================] - 1s 939us/step - loss: 0.7988 - accuracy: 0.7292
Epoch 3/100
804/804 [==============================] - 1s 976us/step - loss: 0.7975 - accuracy: 0.7303
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.7965 - accuracy: 0.7320
Epoch 5/100
804/804 [==============================] - 1s 960us/step - loss: 0.7961 - accuracy: 0.7320
Epoch 6/100
804/804 [==============================] - 1s 754us/step - loss: 0.7954 - accuracy: 0.7315
Epoch 7/100
804/804 [==============================] - 1s 738us/step - loss: 0.7956 - accuracy: 0.7330
Epoch 8/100
804/804 [==============================] - 1s 754us/step - loss: 0.7949 - accuracy: 0.7324
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.7946 - accuracy: 0.7337
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0

In [15]:
# Evaluating the model
model_loss , model_accuracy = nn_model.evaluate(X_test_scaled , y_test , verbose = 2)
print(f'loss: {model_loss} , Accuracy: {model_accuracy}')

268/268 - 0s - loss: 0.7988 - accuracy: 0.7258
loss: 0.7988148331642151 , Accuracy: 0.7258309125900269


In [16]:
# Export results to HDF5 file named AlphabetSoupCharity.h5
nn_model.save('AlphabetSoupCharity_attempt1.h5')

Using the poisson loss function actually decreased accuracy of our model from .726 in predict.ipynb to .725.

## Optimization attempt 2: Increasing the number of epochs
Increasing the number of epochs may give the model more time to converge , I will increase the number of epochs by 50 and see what the results are.

In [17]:
# Compiling  the model
nn_model.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])

# Fit the model to the training data
fit_model = nn_model.fit(X_train_scaled , y_train , epochs = 150)

Epoch 1/150
804/804 [==============================] - 1s 891us/step - loss: 0.5377 - accuracy: 0.7400
Epoch 2/150
804/804 [==============================] - 1s 999us/step - loss: 0.5372 - accuracy: 0.7412
Epoch 3/150
804/804 [==============================] - 1s 819us/step - loss: 0.5370 - accuracy: 0.7409
Epoch 4/150
804/804 [==============================] - 1s 785us/step - loss: 0.5365 - accuracy: 0.7406
Epoch 5/150
804/804 [==============================] - 1s 798us/step - loss: 0.5368 - accuracy: 0.7405
Epoch 6/150
804/804 [==============================] - 1s 819us/step - loss: 0.5362 - accuracy: 0.7407
Epoch 7/150
804/804 [==============================] - 1s 804us/step - loss: 0.5366 - accuracy: 0.7407
Epoch 8/150
804/804 [==============================] - 1s 811us/step - loss: 0.5365 - accuracy: 0.7405
Epoch 9/150
804/804 [==============================] - 1s 799us/step - loss: 0.5366 - accuracy: 0.7405
Epoch 10/150
804/804 [==============================] - 1s 805us/step - l

In [18]:
# Evaluating the model
model_loss , model_accuracy = nn_model.evaluate(X_test_scaled , y_test , verbose = 2)
print(f'loss: {model_loss} , Accuracy: {model_accuracy}')

268/268 - 0s - loss: 0.5717 - accuracy: 0.7258
loss: 0.5716862082481384 , Accuracy: 0.7258309125900269


In [19]:
# Export results to HDF5 file named AlphabetSoupCharity.h5
nn_model.save('AlphabetSoupCharity_attempt2.h5')

Increasing the epochs resulted in a very similar accuracy value as the original model in predict.py. This means that our accuracy loss was not due to a a=lack of convergence.

## Optimization Attempt 3: Increasing the number of hidden neurons
The number of layers (depth) and the number of nodes (width) effect the capacity of the Neural Network. In other words the amount of layers and nuerons we pick for our model can effect how well it fits the data. Too few and we risk underfitting , too many and it becomes a problem of overfitting. In this attempt I am assuming that my combination of layers (2) and number of chosen neurons for each layer (43) is too few and my model is underfitting.


In [12]:
import tensorflow as tf
# Creating our neural network
nn_model = tf.keras.models.Sequential()

# Creating our first Dense Layer
nn_model.add(tf.keras.layers.Dense(units=43 ,activation= 'relu' , input_dim = 43 )) # there are 43 features that will serve as inputs

# Adding our second layer
nn_model.add(tf.keras.layers.Dense(units=43 , activation= 'relu' , input_dim = 43))

# Creating the output layer
nn_model.add(tf.keras.layers.Dense(units = 1 , activation = 'sigmoid'))

# Check the structure of the model
nn_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 43)                1892      
_________________________________________________________________
dense_10 (Dense)             (None, 43)                1892      
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 44        
Total params: 3,828
Trainable params: 3,828
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compiling  the model
nn_model.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])

# Fit the model to the training data
fit_model = nn_model.fit(X_train_scaled , y_train , epochs = 100)

Epoch 1/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5745 - accuracy: 0.7203
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5565 - accuracy: 0.7310
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5537 - accuracy: 0.7303
Epoch 4/100
804/804 [==============================] - 1s 981us/step - loss: 0.5517 - accuracy: 0.7314
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5506 - accuracy: 0.7333
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5502 - accuracy: 0.7325
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5489 - accuracy: 0.7318
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5485 - accuracy: 0.7331
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5483 - accuracy: 0.7340
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5474 - accu

In [10]:
# Evaluating the model
model_loss , model_accuracy = nn_model.evaluate(X_test_scaled , y_test , verbose = 2)
print(f'loss: {model_loss} , Accuracy: {model_accuracy}')

268/268 - 0s - loss: 0.5526 - accuracy: 0.7255
loss: 0.5525734424591064 , Accuracy: 0.7254810333251953


In [11]:
# Export results to HDF5 file named AlphabetSoupCharity.h5
nn_model.save('AlphabetSoupCharity_attempt3.h5')

Doubling the number of hidden nodes in our network worsened the original score albeit very minimally. I was curious to see if my problem was one of over fitting so I then procedeed to halve my hidden nodes and that seemed to perform just as well as doubling the nodes. From the three attempts I have made it looks like the issue does not lie in the architecture of the model , but in the quality of my input data. In the future , Ceiling analysis may prove useful in pinpoint the pain points of my models.
https://rossbulat.medium.com/ceiling-analysis-in-deep-learning-and-software-development-8bc41e59364a